##  Decorators

In [2]:
def decorate1(func):
    def inner():
        print("apply decorate1")
        func()
    return inner

@decorate1
def hello():
    print("hello, world!")
        
hello()

apply decorate1
hello, world!


In [3]:
%reset -sf 
def decorate1(func):
    def inner():
        print("apply decorate1")
        func()
    return inner

def hello():
    print("hello, world!")

#A decorator in python is a function that have as an argument another function.
#It defines an inner function that has the same argument of the function that will be used as argument for decorator
#e.g. inner and hello have the same amount of arguments.
#functions are objects in python so we can return them. 
#the below call is the same as @decorate

hello = decorate1(hello)
hello()

apply decorate1
hello, world!


In [4]:
def decorate2(func):
    def inner():
        print("apply decorate2")
        func()
    return inner

@decorate1
@decorate2
def hello12():
    print("hello, world!")
hello12() # same as hello12 = decorate1(decorate2(hello12))

apply decorate1
apply decorate2
hello, world!


In [5]:
@decorate2
@decorate1
def hello21():
    print("hello, world!") 
hello21() # same as hello21 = decorate2(decorate1(hello21))

apply decorate2
apply decorate1
hello, world!


###  How to pass arguments to the inner function

In [1]:
#adapted from Fluent Python
import functools
def args_to_string(*args,**kw):
    arg_str = ()
    if args:
        arg_str += (','.join(str(arg) for arg in args)),
    if kw:
        arg_str += (', '.join(('{0}={1}'.format(k,v) for k,v in kw.items()))),
    return ','.join(a for a in arg_str)

import time
def time_this(func):
    def decorated(*args,**kw):
        t0 = time.perf_counter()
        result = func(*args,**kw)
        t1 = time.perf_counter()
        name = func.__name__
        arg_str = args_to_string(*args,**kw)
        #print('{0}({1}): [{2}]'.format(name, arg_str,t1-t0))
        #print('{}({}): [{}]'.format(name, arg_str,t1-t0))
        print('%s(%s): [%0.8f s]' % (name, arg_str, t1-t0))
        return result
    return decorated

@time_this
def wait(seconds):
    time.sleep(seconds)

#This one is a parametrized decorator: you can pass argument to it. Now we are using the default value.
#It caches the argument (as key) and the result (as value) of the function you are calling.
#This means that you don't go through the computation once you've already done it.

@functools.lru_cache() # <-- note () # parametrized decorators
@time_this
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

@time_this
def sum(a,b):
    return a+b

@time_this
def dummy(*args, **kw):
    a = args
    b = kw

wait(0.3)
factorial(10)
sum(4,5)
dummy('pos', 'second', a='a', b='b')

wait(0.3): [0.30065370 s]
factorial(1): [0.00000530 s]
factorial(2): [0.00014030 s]
factorial(3): [0.00024630 s]
factorial(4): [0.00032940 s]
factorial(5): [0.00053610 s]
factorial(6): [0.00098630 s]
factorial(7): [0.00107220 s]
factorial(8): [0.00114200 s]
factorial(9): [0.00127890 s]
factorial(10): [0.00142230 s]
sum(4,5): [0.00000530 s]
dummy(pos,second,a=a, b=b): [0.00000600 s]


In [2]:
#You are not actually calling the function factorial: you are yielding the result which is already in the cache.
#That's why you don't get the decorator output also
factorial(10)

3628800

In [3]:
import time
def parametrized_time_this(check=True):
    def decorator(func):
        if not check:
            return func
        def decorated(*args,**kw):
            t0 = time.perf_counter()
            result = func(*args,**kw)
            t1 = time.perf_counter()
            name = func.__name__
            arg_str = args_to_string(*args,**kw)
            print('%s(%s): [%0.8f s]' % (name, arg_str, t1-t0))
            return result
        return decorated
    return decorator # <-- returns the actual decorator
   
#We actually have a function here: the function returns a decorator. This way we can have multiple
#behaviour of a decorator (such as don't decorate anything)
@parametrized_time_this(True)
def wait(seconds):
    print('going to sleep for', seconds,'seconds')
    time.sleep(seconds)
    print('woke up!')

wait(0.4)

going to sleep for 0.4 seconds
woke up!
wait(0.4): [0.40123030 s]


### Decorators as function objects

In [4]:
import time
class TimeThis():
    def __init__(self, func):           # <--
        self._func = func               # <--
        
    def __call__(self, *args, **kw):
        t0 = time.perf_counter()
        result = self._func(*args,**kw) # <--
        t1 = time.perf_counter()
        name = self._func.__name__      # <--
        arg_str = args_to_string(*args,**kw)
        print('%s(%s): [%0.8f s]' % (name, arg_str, t1-t0))
        return result

@TimeThis
def wait(seconds):
    print('going to sleep for', seconds,'seconds')
    time.sleep(seconds)
    print('woke up!')

wait(0.4)

going to sleep for 0.4 seconds
woke up!
wait(0.4): [0.40106310 s]


In [7]:
class ParametrizedTimeThis():
    def __init__(self, check=True):
        self.check = check
    def __call__(self,func):
        if self.check:
            #return TimeThis(func)
            @TimeThis
            def wrapper(*args,**kwargs):
                return func(*args,**kwargs)
            return wrapper
        return func
        
@ParametrizedTimeThis(True)
def wait(seconds):
    print('going to sleep for', seconds,'seconds')
    time.sleep(seconds)
    print('woke up!')

wait(0.4)

@ParametrizedTimeThis(False)
def waitMe(seconds):
    print("I'll be a long time gone")
    time.sleep(seconds)
    print('joking, LOL')
    
waitMe(0.7)

going to sleep for 0.4 seconds
woke up!
wrapper(0.4): [0.40174840 s]
I'll be a long time gone
joking, LOL


In [8]:
PTT = ParametrizedTimeThis(True)
#Apply the sticky note PTT

@PTT
def dummy(*args,**kw):
    pass

dummy(0.4)

wrapper(0.4): [0.00000460 s]
